In [ ]:
import csv, os
from tqdm import tqdm
import numpy as np

In [ ]:
directory_path = "/content/include-50"

data = []

for folder_name in os.listdir(directory_path):
    folder_path = os.path.join(directory_path, folder_name)
    if os.path.isdir(folder_path):
        for subfolder_name in os.listdir(folder_path):
            subfolder_path = os.path.join(folder_path, subfolder_name)
            if os.path.isdir(subfolder_path):
                for filename in os.listdir(subfolder_path):
                    if filename.endswith(".MOV"):
                        video_path = os.path.join(subfolder_path, filename)
                        gloss = subfolder_name.split(". ")[1]
                        data.append({"gloss": gloss, "video_path": video_path})

with open("output.csv", "w", newline="") as csvfile:
    fieldnames = ["gloss", "video_path"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for item in data:
        writer.writerow(item)

print("CSV file saved successfully!")

In [ ]:
npy_dir = '/content/output'
os.makedirs(npy_dir, exist_ok=True)

for i in tqdm(range(len(data)), ncols=100):
    try:
        gloss_name = data[i]['gloss']
        npy1_dir = os.path.join(npy_dir, gloss_name)
        os.makedirs(npy1_dir, exist_ok=True)

        npy_path = os.path.join(npy1_dir, f'{i}.npy')
        if os.path.exists(npy_path):
            continue

        video_path = data[i]['video_path']

        try:
            video_landmarks = get_video_landmarks(video_path)
            np.save(npy_path, video_landmarks)

        except Exception as e:
            print(f"\nError encoding {video_path}\n{e}")
            continue
        clear_output(wait=True)

    except KeyboardInterrupt:
        print("\nLoading process interrupted by user.")
        break

In [ ]:
landmarks_dict = {}
npy_dir = "/content/output/"
for filename in os.listdir(npy_dir):
    if filename.endswith('.npy'):
        key = filename.split('.')[0]
        landmarks = np.load(os.path.join(npy_dir, filename), allow_pickle=True)
        landmarks_dict[key] = landmarks

np.savez_compressed('/content/landmarks.npz', **landmarks_dict)